In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
df_tot = pd.read_csv("data/final.csv")

In [ ]:
# per rispondere alle mie domande mi basta avere gli elements, quindi elimino le rige relative ai components
print("initial df.shape:",df_tot.shape)
df = df_tot.drop(df_tot[df_tot.section == "components"].index)
print("post cut df.shape:",df.shape)

In [ ]:
# I giudici tendono a dare voti più alti ai propri connazionali?
# per rispondere mi creo una tabella con la media dei voti che ciascun giudice ha dato
# a ciascun atleta e le nazionalità di entrambi
df.head(1)

In [ ]:
# elimino le colonne che non mi servono più
df.drop('total_segment_score', axis=1, inplace=True)
df.drop('total_component_score', axis=1, inplace=True)
df.drop('total_element_score', axis=1, inplace=True)
df.drop('section', axis=1, inplace=True)
df_q2 = df.drop('event', axis=1)
df_q2.drop('program', axis=1, inplace=True)
df_q2.drop('performance_id', axis=1, inplace=True)
df_q2.drop('rank', axis=1, inplace=True)
df_q2.drop('total_deductions', axis=1, inplace=True)
df_q2.drop('aspect_id', axis=1, inplace=True)
df_q2.drop('aspect_desc', axis=1, inplace=True)
df_q2.drop('element_order', axis=1, inplace=True)
df_q2.drop('element_base_value', axis=1, inplace=True)
df_q2.drop('aspect_final_score', axis=1, inplace=True)
df_q2.drop('judge_name', axis=1, inplace=True)
df_q2.head(2)

In [ ]:
# creo una tabella che abbina a ciascun atleta il voto medio ricevuto dai giudici
# raggruppato per nazionalità dei giudici
# esempio: la media dei voti dei giudici canadesi ad Adam RIPPON è 1.5 (riga 2)
medie = df_q2.groupby(['athlete_name','athlete_nation','judge_nation'])['judge_score'].agg(['mean']).reset_index()

In [ ]:
medie.head(5)

In [ ]:
# creo la tabella atleti_voti, che alla fine sarà formata nel seguente modo:
# - athlete_name: nome dell'atleta
# - athlete_nation: nazionalità dell'atleta
# - compatriot_score: la media dei voti dati dai giudici connazionali dell'atleta
# - mean_other_nations: la media dei voti ricevuti dall'atleta, esclusi i voti dati dai connazionali
atleti_voti = pd.DataFrame(columns=['athlete_name','athlete_nation','compatriot_score'])
atleti_voti.head()

In [ ]:
# create a colum to index rows to be deleted
medie['tbd'] = False
# popolo la tabella atleti_voti con i voti dei connazionali
for ind in medie.index:
    if medie['athlete_nation'][ind] == medie['judge_nation'][ind]:
        medie.at[ind,'tbd'] = True
        atleti_voti.loc[len(atleti_voti.index)] = [medie['athlete_name'][ind],medie['athlete_nation'][ind], medie['mean'][ind]] 

In [ ]:
atleti_voti.head()

In [ ]:
# per fare la media dei voti ricevuti dall'atleta, esclusi i voti dati dai connazionali,
# elimino le colonne athlete_nation e judge_nation  e elimino le righe in cui c'è il voto dei compatrioti
medie.drop('athlete_nation', axis=1, inplace=True)
medie.drop('judge_nation', axis=1, inplace=True)
medie.drop(medie[medie.tbd].index, inplace=True)
medie.drop('tbd', axis=1, inplace=True)
medie.rename(columns={'mean': 'score'}, inplace=True)

In [ ]:
medie.head()

In [ ]:
medie = medie.groupby('athlete_name')['score'].agg(['mean']).reset_index()

In [ ]:
medie.head()

In [ ]:
# facendo la join tra le due tabelle, popolo la colonna mean_other_nations di atleti_voti
# (vogliamo considerare solo gli atleti che sono stati giudicati da connazionali, scartiamo gli altri)
print('atleti_voti.shape: ',atleti_voti.shape)
print('medie.shape: ',medie.shape)
atleti_voti = atleti_voti.merge(medie,
                how='inner',
                on='athlete_name')
medie.rename(columns={'mean': 'mean_other_nations'}, inplace=True)
print('atleti_voti.shape after: ',atleti_voti.shape)

In [ ]:
atleti_voti.head()

In [ ]:
atleti_voti.shape

In [ ]:
# per poter fare il barplot devo fare quanto segue
tidy = atleti_voti.drop('athlete_nation', axis=1)
tidy = tidy.melt(id_vars='athlete_name').rename(columns=str.title)

In [ ]:
# adesso i voti dati ad un atleta sono salvati in questo modo
tidy.loc[tidy['Athlete_Name'] == 'Adam RIPPON']

In [ ]:
# voglio fare la stessa cosa ma per nazioni, invece che per atleti
# la tabella nazioni_voti sarà come segue:
# - nation: nazionalità considerata
# - compatriot_score: la media dei voti dati agli atleti di nation dai giudici connazionali
# - mean_other_nations: la media dei voti ricevuti dagli atleti di nation, esclusi i voti dati dai connazionali
nazioni_voti = pd.DataFrame(columns=['nation','compatriot_score'])
nazioni_voti.head()

In [ ]:
# creo una tabella che abbina a ciascun atleta il voto medio ricevuto dai giudici
# raggruppato per nazionalità dei giudici
# esempio: la media dei voti dei giudici canadesi ad Adam RIPPON è 1.5 (riga 2)
medie = df_q2.groupby(['athlete_nation','judge_nation'])['judge_score'].agg(['mean']).reset_index()

In [ ]:
medie.head()

In [ ]:
# create a colum to index rows to be deleted
medie['tbd'] = False
# popolo la tabella nazioni_voti con i voti dei connazionali
for ind in medie.index:
    if medie['athlete_nation'][ind] == medie['judge_nation'][ind]:
        medie.at[ind,'tbd'] = True
        nazioni_voti.loc[len(nazioni_voti.index)] = [medie['athlete_nation'][ind], medie['mean'][ind]] 

In [ ]:
medie.head()

In [ ]:
nazioni_voti.head()

In [ ]:
# per fare la media dei voti ricevuti da una nazione, esclusi i voti dati dai connazionali,
# elimino le colonne udge_nation e elimino le righe in cui c'è il voto dei compatrioti
medie.drop('judge_nation', axis=1, inplace=True)
medie.drop(medie[medie.tbd].index, inplace=True)
medie.drop('tbd', axis=1, inplace=True)
medie.rename(columns={'mean': 'score'}, inplace=True)

In [ ]:
medie.head()

In [ ]:
medie = medie.groupby('athlete_nation')['score'].agg(['mean']).reset_index()

In [ ]:
medie.head(1)

In [ ]:
nazioni_voti.head(1)

In [ ]:
# facendo la join tra le due tabelle, popolo la colonna mean_other_nations di nazioni_voti
# (vogliamo considerare solo le nazioni che hanno ricevuto voti da giudici di quella nazione, scartiamo gli altri)
print('nazioni_voti.shape: ',nazioni_voti.shape)
print('medie.shape: ',medie.shape)
nazioni_voti = nazioni_voti.merge(medie,
                how='inner',
                left_on='nation',
                right_on = 'athlete_nation')
nazioni_voti.rename(columns={'mean': 'mean_other_nations'}, inplace=True)
print('nazioni_voti.shape after: ',nazioni_voti.shape)

In [ ]:
nazioni_voti.drop('athlete_nation', axis=1, inplace=True)
nazioni_voti.head()

In [ ]:
# per poter fare il barplot devo fare quanto segue
tidy = nazioni_voti.melt(id_vars='nation').rename(columns=str.title)

In [ ]:
tidy.head()

In [ ]:
# adesso i voti dati ad un atleta sono salvati in questo modo
tidy.loc[tidy['Nation'] == 'AUS']

In [ ]:
tidy.head()

In [ ]:
nazioni_voti.head()

In [ ]:
# faccio il grafico con le proporzioni della slide di power point

fig = make_subplots(shared_yaxes=True, shared_xaxes=True)
fig.add_bar(x=nazioni_voti['nation'],
            y=nazioni_voti['mean_other_nations'],
            width=0.75,
            #marker=dict(color='#F9A311'),
            name='Other nationalities',
            #hovertemplate='#F9A311')
           )
fig.add_bar(x=nazioni_voti['nation'],
            y=nazioni_voti['compatriot_score'],
            width=0.25,
            #marker=dict(color='#1167F9'),
            name='Compatriots')
fig.update_layout(#general features
                  barmode='overlay',
                  width=1315,
                  height=700,
                  plot_bgcolor='#E9F1FA',
                  paper_bgcolor='#E9F1FA',
    
                  #margins
                  margin=dict(l=60, r=60, t=120, b=80),
    
                  #legend
                  legend=dict(yanchor="top",
                              xanchor="center",
                              orientation="h",
                              y=1.01,
                              x=0.5),
                  #title
                  title={'text': "<b>Do the judges support compatriot athletes?</b>",
                          'y':0.97,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'},
                  title_font={'family': 'DejaVu Sans',
                               'size': 24,
                               'color': '#543810'},
                  #general font
                  font={'family': 'DejaVu Sans',
                        'size': 14,
                        'color': '#543810'})


# sottotitolo riga 1
fig.add_annotation(text = ("Comparison between the average points given to a Nation’s athletes by their compatriots"),
                   showarrow=False,
                   x = 0, 
                   y = -0.14, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='center', 
                   yanchor='top', 
                   xshift=580, 
                   yshift=640, 
                   font=dict(size=18, color="#543810", family='DejaVu Sans'), 
                   align="center")

# sottotitolo riga 2
fig.add_annotation(text = (""),
                   showarrow=False,
                   x = 0, 
                   y = -0.14, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='center', 
                   yanchor='top', 
                   xshift=580, 
                   yshift=620, 
                   font=dict(size=18, color="#543810", family='DejaVu Sans'), 
                   align="center")


# aggiungi autore e origine dati
fig.add_annotation(text = "data source: https://github.com/BuzzFeedNews/2018-02-figure-skating-analysis",
                   showarrow=False,
                   x = 0, 
                   y = -0.11, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='left', 
                   yanchor='bottom', 
                   xshift=-1, 
                   yshift=-5, 
                   font=dict(size=10, color="#543810", family='DejaVu Sans'), 
                   align="left")
fig.add_annotation(text = "plot author: Cecilia Zagni",
                   showarrow=False,
                   x = 0, 
                   y = -0.14, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='left', 
                   yanchor='bottom', 
                   xshift=-1, 
                   yshift=-5, 
                   font=dict(size=10, color="#543810", family='DejaVu Sans'), 
                   align="left")
#titolo asse y
fig.add_annotation(text = "Score",
                   showarrow=False,
                   x = 0, 
                   y = -0.15, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='left', 
                   yanchor='top', 
                   xshift=-50, 
                   yshift=575, 
                   font=dict(size=16, color="#543810", family='DejaVu Sans'), 
                   align="left")

fig.update_traces(marker_line_width = 0)
fig.update_xaxes(tickangle=0)

In [ ]:
# ordino le nazioni per voto altri giudici decrescente
nazioni_voti.sort_values(by=['mean_other_nations'],ascending=False,inplace=True)

In [ ]:
# faccio il grafico con le proporzioni della slide di power point

fig = make_subplots(shared_yaxes=True, shared_xaxes=True)
fig.add_bar(x=nazioni_voti['nation'],
            y=nazioni_voti['mean_other_nations'],
            width=0.75,
            #marker=dict(color='#F9A311'),
            name='Other nationalities',
            #hovertemplate='#F9A311')
           )
fig.add_bar(x=nazioni_voti['nation'],
            y=nazioni_voti['compatriot_score'],
            width=0.25,
            #marker=dict(color='#1167F9'),
            name='Compatriots')
fig.update_layout(#general features
                  barmode='overlay',
                  width=1315,
                  height=700,
                  plot_bgcolor='#E9F1FA',
                  paper_bgcolor='#E9F1FA',
    
                  #margins
                  margin=dict(l=60, r=60, t=120, b=80),
    
                  #legend
                  legend=dict(yanchor="top",
                              xanchor="center",
                              orientation="h",
                              y=1.01,
                              x=0.5),
                  #title
                  title={'text': "<b>Do the judges support compatriot athletes?</b>",
                          'y':0.97,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'},
                  title_font={'family': 'DejaVu Sans',
                               'size': 24,
                               'color': '#543810'},
                  #general font
                  font={'family': 'DejaVu Sans',
                        'size': 14,
                        'color': '#543810'})


# sottotitolo riga 1
fig.add_annotation(text = ("Comparison between the average points given to a Nation’s athletes by their compatriots"),
                   showarrow=False,
                   x = 0, 
                   y = -0.14, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='center', 
                   yanchor='top', 
                   xshift=580, 
                   yshift=640, 
                   font=dict(size=18, color="#543810", family='DejaVu Sans'), 
                   align="center")

# sottotitolo riga 2
fig.add_annotation(text = (""),
                   showarrow=False,
                   x = 0, 
                   y = -0.14, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='center', 
                   yanchor='top', 
                   xshift=580, 
                   yshift=620, 
                   font=dict(size=18, color="#543810", family='DejaVu Sans'), 
                   align="center")


# aggiungi autore e origine dati
fig.add_annotation(text = "data source: https://github.com/BuzzFeedNews/2018-02-figure-skating-analysis",
                   showarrow=False,
                   x = 0, 
                   y = -0.11, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='left', 
                   yanchor='bottom', 
                   xshift=-1, 
                   yshift=-5, 
                   font=dict(size=10, color="#543810", family='DejaVu Sans'), 
                   align="left")
fig.add_annotation(text = "plot author: Cecilia Zagni",
                   showarrow=False,
                   x = 0, 
                   y = -0.14, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='left', 
                   yanchor='bottom', 
                   xshift=-1, 
                   yshift=-5, 
                   font=dict(size=10, color="#543810", family='DejaVu Sans'), 
                   align="left")
#titolo asse y
fig.add_annotation(text = "Score",
                   showarrow=False,
                   x = 0, 
                   y = -0.15, 
                   xref='paper', 
                   yref='paper', 
                   xanchor='left', 
                   yanchor='top', 
                   xshift=-50, 
                   yshift=575, 
                   font=dict(size=16, color="#543810", family='DejaVu Sans'), 
                   align="left")

fig.update_traces(marker_line_width = 0)
fig.update_xaxes(tickangle=0)